In [ ]:
import pandas as pd
import win32com.client
from datetime import datetime

원본 엑셀 파일을 오픈한 후 '종합' 시트에서 영역을 선택, 클립보드로 복사한 후 실행해 주세요.

1. 종합 시트 내용을 불러와 df로 변환하고, 통계를 계산해 변수에 저장한다.

In [ ]:
df = pd.read_clipboard()

#성별 인원수 뽑아오기
df_gr = df.groupby("성별")
gender_cnt = df_gr.count()
male_cnt = gender_cnt.loc["남", "성명"]
female_cnt = gender_cnt.loc["여", "성명"]
# male_cnt, female_cnt

#평균연령, 최소연령, 최대연령 뽑아오기
df_mean = df.mean()
age_mean = df_mean["연령"]
age_mean = round(age_mean, 1)
df_min = df.min()
age_min = df_min["연령"]
age_max = df.max()["연령"]

#회사 개수, 교육생 인원 뽑아오기
# df["회사명"] = df["회사명"].str.strip()
df["회사명"] = df["회사명"].str.replace(" ", "")
df["회사명"] = df["회사명"].str.replace("\xa0", "") #추가된 코드
company_list = df["회사명"].unique()
company_cnt = len(company_list)
student_cnt = df.count()["성명"]

#통계 내용을 문자열로 만들어 변수에 저장
stats_str = "· 성비 : 남 {}, 여 {} / 평균 연령 : {}세({}-{}세) / {}개社 {}명 입과".format(
    male_cnt, female_cnt, age_mean, age_min, age_max, company_cnt, student_cnt
)

2. 회사별 교육생 수를 파악하고 건제순으로 문자열로 만들어 변수에 저장한다.

In [ ]:
df_gr_company = df.groupby("회사명")
df_gr_company.count()["성명"]
cnt_by_company = df_gr_company.count()["성명"].to_dict()

#아래는 추가된 코드
company_list_by_order = pd.read_excel("회사목록(건제순).xlsx")
com_order = {}
cnt = 0
for item in company_list_by_order["국내그룹사"]:
    cnt += 1
    com_order[item] = cnt
company_list_ordered = pd.DataFrame({"회사명" : company_list})
company_list_ordered["순서"] = company_list_ordered["회사명"].map(com_order)
company_list_ordered["교육생수"] = company_list_ordered["회사명"].map(cnt_by_company)
company_list_ordered.sort_values(by="순서", inplace=True, ascending=True)

cnt_by_company_str = ""
for item in company_list_ordered.iterrows():
    cnt_by_company_str = cnt_by_company_str + "{} {}명, ".format(item[1]["회사명"], item[1]["교육생수"])
cnt_by_company_str = cnt_by_company_str[:-2]

3. 회사 건제순으로 정렬된 교육생 명단을 만든다.

In [ ]:
df["구분"] = 0
df_result = df[["구분", "성명", "조 편성", "회사명", "연령", "성별", "휴대전화번호"]]
df_result.rename(columns={
    "성명" : "이름",
    "조 편성" : "조",
    "회사명" : "회사",
    "휴대전화번호" : "휴대폰"
}, inplace=True)
df_result["건제순"] = 0
df_result["건제순"] = df_result["회사"].map(com_order)
df_result.sort_values(by=["건제순", "이름"], inplace=True)
gubun_list = []
for i in range(1, len(df_result)+1):
    gubun_list.append(i)
df_result["구분"] = gubun_list

4. 엑셀 파일에 반영하고 '교육생 명단(결과).xlsx' 파일로 저장한다.

In [ ]:
excel = win32com.client.Dispatch("Excel.Application")
excel.Workbooks.Open("교육생 명단(양식).xlsx")
sheet = excel.ActiveSheet
sheet.Cells(2, 2).Value = stats_str
sheet.Cells(3, 2).Value = cnt_by_company_str
start_row_no = 5
for item in df_result.iterrows():
    sheet.Cells(start_row_no, 2).Value = item[1]["구분"]
    sheet.Cells(start_row_no, 3).Value = item[1]["이름"]
    sheet.Cells(start_row_no, 4).Value = item[1]["조"]
    sheet.Cells(start_row_no, 5).Value = item[1]["회사"]
    sheet.Cells(start_row_no, 6).Value = item[1]["연령"]
    sheet.Cells(start_row_no, 7).Value = item[1]["성별"]
    sheet.Cells(start_row_no, 8).Value = item[1]["휴대폰"]
    start_row_no = start_row_no + 1
now = datetime.now()
now_str = datetime.strftime(now, "%Y%m%d_%H%M%S")

excel.ActiveWorkbook.SaveAs("교육생 명단(결과)_{}.xlsx".format(now_str))